In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import spacy
import torch
from spacy.language import Language
from spacy import displacy
import time
import glob
import re
import math
import statistics
import os
import json
import calendar
import holidays
from pathlib import Path
from datetime import date
from datetime import datetime
import pandas as pd
import numpy as np
import collections
import hashlib
from dateutil.parser import parse
import shutil
import ast
from io import StringIO
import requests
import glob

In [2]:
import spacy
from spacy.language import Language
from spacy import displacy
import time

@Language.component("newsent")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        #print(token.text, token.text in ("’s", "'s"))
        if token.text.upper() in (";", "--", "\n\n", "\n", "QUARTERLY", "STORY", "\n\n\n\n", "\n\n\n"):
            #print("Detected:", token.text)
            doc[token.i].is_sent_start = True
    return doc

#spacy.require_gpu()
nlp = spacy.load("../../Summary/NER/RelateEntity/train/model-best-local")
nlp.add_pipe('sentencizer')
nlp.add_pipe('newsent', name="newsent", last=True)

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy_transformers\pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<function __main__.set_custom_boundaries(doc)>

In [3]:
def isScanningRqd(pfile):
    with open(pfile, 'r', encoding = "utf-8") as fp:
        for l_no, line in enumerate(fp):
            if "NOPAD***" in line:
                return False
    return True

In [4]:
def getSentences(inputfile, nlp, text=None):
    if(not text):
        with open(inputfile, 'r', encoding="utf-8") as f:
            text = f.read()

    doc = nlp(text)
    sentences = [str(sent).strip() for sent in doc.sents]

    #print(len(sentences))
    return(sentences)

In [5]:
rplStr = ["PG*** ", "ED*** ", "SCHQ*** ", "SCBQ*** ", "SCBF*** ", "SCHF*** ", "SCG*** ", "GF*** ", "GQ*** ", "SC*** ", "NOPAD*** "]

def preProcessSent(line):
    for s in rplStr:
        line = line.replace(s, "")
    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
        return(line, False)
    line = line.replace("\n", "")
    if(line == None or line == ""):
        return(line, False)
    return(line, True)

In [6]:
import hashlib
def getHash(sent):
    hash_object = hashlib.sha1(sent.upper().encode("UTF-8"))
    hex_dig = hash_object.hexdigest()
    return(hex_dig)

In [7]:
def getRelationFromER(sym):
    entityFile = "../../Summary/entities/"+sym+"-ENTITIES.json"
    print(entityFile)
    #search = "Q\d+-[0-9][0-9][0-9][0-9]|ALL-[0-9][0-9][0-9][0-9]"
    search = "Q\d+-[0-9][0-9][0-9][0-9]"
    print(search)
    cnt = 0
    sentER = dict()
    with open(entityFile, encoding="utf-8") as f:
        entity = json.load(f)
    if(entity):
        for key in entity[sym].keys():
            if(re.search(search, key)):
                if("ALL" not in key):
                    cnt = cnt + 1
                #if(cnt == 1):
                #    continue
                if(cnt > 5):
                    break
                for ent in entity[sym][key]:
                    if("PREV" not in ent and type(entity[sym][key][ent]) == dict and "RELATION" in entity[sym][key][ent] and len(entity[sym][key][ent]["RELATION"]) > 0):
                        for r in entity[sym][key][ent]["RELATION"]:
                            newent = ent.replace("-GUIDEFULL","")
                            newent = newent.replace("-GUIDE","")
                            newent = newent.replace("-FULLYEAR","")
                            newent = re.sub("-"+search, "", newent)
                            sent = r.split(":::")[2]
                            sent = sent.strip()
                            hexDig = getHash(sent)
                            relation = "METRIC:"+newent+","
                            if hexDig not in sentER:
                                sentER[hexDig] = dict()
                            else:
                                if("RELATION" in sentER[hexDig] and relation in sentER[hexDig]["RELATION"][0]):
                                    print("SAME RELATION " + relation + " ALREADY PRESENT FOR " + sent)
                                    continue
                            if("TEXT-METRICTYPE" in entity[sym][key][ent] and entity[sym][key][ent]["TEXT-METRICTYPE"] == "OPMETRIC"):
                                relation = relation + "TYPE:OPERATIONAL,"
                            else:
                                relation = relation + "TYPE:REGULAR,"
                            if("RTEXT-MONEY" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                money = entity[sym][key][ent]["RTEXT-MONEY"]
                                money = money.replace("MN", " MN")
                                money = money.replace("BN", " BN")
                                
                                rmoney = entity[sym][key][ent]["RTEXT-RANGEM1"]
                                rmoney = rmoney.replace("MN", " MN")
                                rmoney = rmoney.replace("BN", " BN")
                                
                                relation = relation + "MONEY:"+money+",MONEY:"+rmoney+",RELATION:KVRANGE"
                            elif("RTEXT-CD" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                cd = entity[sym][key][ent]["RTEXT-CD"]
                                cd = cd.replace("MN", " MN")
                                cd = cd.replace("BN", " BN")
                                
                                rcd = entity[sym][key][ent]["RTEXT-RANGEM1"]
                                rcd = rcd.replace("MN", " MN")
                                rcd = rcd.replace("BN", " BN")
                                
                                relation = relation + "CD:"+entity[sym][key][ent]["RTEXT-CD"]+",CD:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE"
                            elif("RTEXT-PCT" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",PCT:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE"
                            elif("RTEXT-MONEY" in entity[sym][key][ent]):
                                money = entity[sym][key][ent]["RTEXT-MONEY"]
                                money = money.replace("MN", " MN")
                                money = money.replace("BN", " BN")
                                relation = relation + "MONEY:"+money+",RELATION:KV"
                            elif("RTEXT-CD" in entity[sym][key][ent]):
                                cd = entity[sym][key][ent]["RTEXT-CD"]
                                cd = cd.replace("MN", " MN")
                                cd = cd.replace("BN", " BN")
                                relation = relation + "CD:"+cd+",RELATION:KV"
                            elif("RTEXT-PCT" in entity[sym][key][ent]):
                                relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",RELATION:KV"
                            #print(key, sent+"\t"+relation, hexDig)
                            sentER[hexDig]["ORIGSENT"] = sent
                            sentER[hexDig]["KEY"] = key
                            #sentER[hexDig]["PROPERTY"] = "REGULAR"
                            if("RELATION" not in sentER[hexDig]):
                                sentER[hexDig]["RELATION"] = list()
                            sentER[hexDig]["RELATION"].append(relation)
    #print(sentER)
    return(sentER)

In [8]:
def getFileSortTimestamp(csym, src="REFINED"):
    if(src == "POSTREFINED"):
        path = "../../Summary/PostRefined"
    else:
        path = "../../Summary/Refined"
    files = glob.glob(path+"/"+csym+"/*")   
    files.sort(reverse=True, key=os.path.getmtime)
    return(files)

In [9]:
def getOrgData(org):
    orgDataPath = "../../Summary/orgData/"+org+".txt"
    file = Path(orgDataPath)
    if file.is_file():
        #print(True)
        with open(orgDataPath) as f:
            data = json.load(f)
        #print(data)
        return data
    return None

In [10]:
def getOrgAttr(orgData, attr):
    if not orgData:
        return None
    asplit = attr.split("|")
    parent = asplit[0]
    if "SOURCE" in orgData[parent]:
        src = orgData[parent]["SOURCE"]
        if src == "YH" or (parent == "ORGPROFILE" and src == "AD"):
            p = orgData
            for i in range(0, len(asplit)):
                if asplit[i] not in p:
                    return None
                p = p[asplit[i]]
            #print(p)
            return(p)
    return None

In [11]:
def getQtrs(org, edate, fynd):
    #print(edate)
    dateObj = datetime.strptime(edate, '%Y-%m-%d')
    #print(dateObj.month)
    emonth = dateObj.month
    eyr = dateObj.year
    k = 1
    qend = fynd
    qstart = 0
    year = eyr
    #print(year)
    qtrs = dict()
    while(k!=5):
        #print(k)
        qstart = (qend + 1)%12
        qend = (qend + 3)%12
        if qstart == 0:
            qstart = 12
        if qend == 0:
            qend = 12
        if((emonth >= qstart and emonth <= qend) or (qstart > qend and (emonth >= qstart or emonth <= qend))):
            #print("CQTR:", "Q"+str(k))
            qtrs["CQTR"] = "Q"+str(k)
            qtrs["GQTR"] = "Q"+str(k) # Guidance qtr is same as current qtr
            qtrs["CYR"] = str(year)
            qtrs["GYR"] = str(year) # Guidance year same as current year
            eqtr = (k-1)%4
            if(eqtr == 0):
                eqtr = 4
            nqtr = (k+1)%4
            if(nqtr == 0):
                nqtr = 4
            pqtr = (k-2)%4
            if(pqtr == 0):
                pqtr = 4
            qtrs["EQTR"] = "Q"+str(eqtr)
            qtrs["PQTR"] = "Q"+str(pqtr)
            #print("EQTR:", "Q"+str(eqtr))
            #print("PQTR:", "Q"+str(pqtr))
            #print("NQTR:", "Q"+str(nqtr))
        #print("Q"+str(k), "START", qstart)
        #print("Q"+str(k), "END", qend)
        qtr = "Q"+str(k)
        if (qtr not in qtrs):
            qtrs[qtr] = dict()
        qtrs[qtr]["START"] = qstart
        qtrs[qtr]["SM"] = calendar.month_abbr[qstart].upper()
        qtrs[qtr]["END"] = qend
        qtrs[qtr]["EM"] = calendar.month_abbr[qend].upper()
        #print("Q"+str(k), "YEAR", year)
        k = k + 1
    
    #print(eyr)   
    if("EQTR" in qtrs and "CQTR" in qtrs and "PQTR" in qtrs):
        eqtrEnd = qtrs[qtrs["EQTR"]]["END"]
        cqtrEnd = qtrs[qtrs["CQTR"]]["END"]
        pqtrEnd = qtrs[qtrs["PQTR"]]["END"]
        #print(eqtrEnd, cqtrEnd, pqtrEnd)
        qtrs["EYR"] = str(eyr)
        qtrs["PYR"] = str(eyr)
        #print(eqtrEnd, cqtrEnd, qtrs[qtrs["CQTR"]]["START"], qtrs[qtrs["CQTR"]]["END"])
        if(eqtrEnd > cqtrEnd):
            if(qtrs[qtrs["CQTR"]]["START"] > qtrs[qtrs["CQTR"]]["END"] and emonth > fynd):
                gyr = year + 1
                qtrs["GYR"] = str(gyr)
                qtrs["EYR"] = str(gyr - 1)
            else:
                qtrs["EYR"] = str(eyr - 1)
                qtrs["PYR"] = str(eyr - 1)
        if(pqtrEnd > eqtrEnd):
            qtrs["PYR"] = str(int(qtrs["EYR"]) - 1)
    if(emonth <= fynd):
        qtrs["CFYR"] = str(year)
    else:
        qtrs["CFYR"] = str(year + 1)
    qtrs["FYR"] = qtrs["CFYR"]
    if(qtrs[qtrs["EQTR"]]["END"] <= fynd and qtrs["EQTR"] > qtrs["CQTR"]):
        qtrs["FYR"] = str(int(qtrs["FYR"]) - 1)
    qtrs["EFYR"] = qtrs["FYR"]
    qtrs["PFYR"] = qtrs["FYR"]
    if(qtrs["PQTR"] > qtrs["EQTR"]):
        qtrs["PFYR"] = str(int(qtrs["FYR"]) - 1)
    qtrs["GFYR"] = qtrs["CFYR"]
    #qtrs["EYR"] = qtrs["FYR"]
    #qtrs["CYR"] = qtrs["CFYR"]
    #qtrs["PYR"] = qtrs["FYR"]
    qtrs["CYR"] = qtrs["GYR"]
    #print(qtrs)
    return (qtrs)

In [12]:
trainDir = "../../Summary/DATA/ENTITYRELATION/Train"
devDir = "../../Summary/DATA/ENTITYRELATION/Dev"
testDir = "../../Summary/DATA/ENTITYRELATION/Test"

def scanER(csym, estr, sentER, infile, outfile=None):
    infilePath = Path(infile)
    if not infilePath:
        print("Input File {} does not exist".format(infile))
        return
    basefile = os.path.basename(infile)
    filename = os.path.splitext(basefile)[0]
    
    outfilePath = None
    of = None
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            return
        
        of = open(outfilePath, "w", encoding = "utf-8")
        of.write("filename\tSentence1\tSentence2\n")
        
    with open(infilePath, "r", encoding = "utf-8") as f:
        line = f.readline()
        #print(line)
        while line:
            #print(line)
            if("ED***" in line):
                break
            elif("CS***" in line or "TBLST***" in line or "TBLET***" in line):
                #print(line+"\t"+"RELATION:None")
                line = f.readline()
                continue
            #elif("PG***" in line or "NOPAD***" in line or ("***" not in line)):
            else:
                sentences = getSentences(None, nlp, line)
                for l in sentences:
                    #print(l)
                    pline, isProcess = preProcessSent(l)
                    if not isProcess:
                        continue
                    text1 = list()
                    text1.append(l)
                    nerl = list()
                    for doc in nlp.pipe(text1, disable=["tagger"]):
                        nerl = ([(ent.text, ent.label_) for ent in doc.ents])
                        #print("NER", nerl)
                    hexDig = getHash(pline)
                    if hexDig in sentER and "KEY" in sentER[hexDig] and sentER[hexDig]["KEY"] == estr:
                        print(l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(sentER[hexDig]["RELATION"]))
                        of.write(filename+"\t"+l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(sentER[hexDig]["RELATION"])+"\n\n\n")
                    else:
                        print(l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(list()))
                        of.write(filename+"\t"+l+"\t"+"NER:"+str(nerl)+"###RELATIONS:"+str(list())+"\n\n\n")
            line = f.readline()
    if(of):
        of.close()

In [13]:
def createTrainingData(csym):
    files = getFileSortTimestamp(csym)
    print(files)
    pfiles = getFileSortTimestamp(csym, "POSTREFINED")
    print(pfiles)

    sentER = getRelationFromER(csym)

    orgData = getOrgData(csym)
    if orgData:
        val = getOrgAttr(orgData, "ORGPROFILE|FiscalYearEnd")

    cnt = 0
    cdate = "2023-06-30"
    for file in files:
        if not isScanningRqd(file):
            continue
        cnt = cnt + 1
        if(cnt > 5):
            break
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        outfile = filename+"_ER.tsv"
        outfile = trainDir + "/" + outfile
        #print(filename)
        edate = filename.split("_")[1]
        rtype = filename.split("_")[0]
        estr = None
        if orgData:
            if(val):
                dobj = datetime.strptime(val, "%B")
                fynd = (dobj.month)
                qtrs = getQtrs(csym, edate, fynd)
                eqtr = qtrs["EQTR"]
                efyr = qtrs["EFYR"]
                estr = eqtr+"-"+efyr
        if(estr):
            nedate = time.strptime(edate, "%Y-%m-%d")
            ncdate = time.strptime(cdate, "%Y-%m-%d")
            if(nedate > ncdate):
                nfile = "../../Summary/PostRefined/"+csym+"/"+basefile
                print(edate, rtype, estr, nfile, outfile)
            else:
                nfile = file
                print(edate, rtype, estr, nfile, outfile)
            scanER(csym, estr, sentER, nfile, outfile)

In [51]:
# Format 1
entPath = "../../Summary/entities/"
files = glob.glob(entPath+"/*-ENTITIES.json")
if(len(files) > 0):
    for file in (files):
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        csym = filename.split("-")[0]
        #print(csym)
        createTrainingData(csym)

['../../Summary/Refined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-08-04_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-05-06_EP_YH.txt']
['../../Summary/PostRefined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-08-04_EP_YH.txt']
../../Summary/entities/APPN-ENTITIES.json
Q\d+-[0-9][0-9][0-9][0-9]
2023-05-09 APPN Q1-2023 ../../Summary/Refined/APPN\APPN_2023-05-09_EP_YH.txt ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv
Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv already exists
2023-08-03 APPN Q2-2023 ../../Summary/Pos

GAAP Net Loss Per Share is $(0.90) in second quarter 2023 @@@	NER:[('GAAP Net Loss Per Share', 'METRIC'), ('$(0.90)', 'MONEY'), ('second quarter', 'CALENDAR'), ('2023', 'YEAR')]###RELATIONS:['METRIC:GAAP-EPS,METRICTYPE:REGULAR,MONEY:-$0.9,RELATION:KV']
Cash And Cash Equivalents is $1616758 T in second quarter 2023 @@@	NER:[('Cash', 'METRIC'), ('Cash Equivalents', 'METRIC'), ('$1616758 T', 'MONEY'), ('second quarter', 'CALENDAR')]###RELATIONS:['METRIC:CASH AND EQUIVALENTS,METRICTYPE:REGULAR,MONEY:$1.62BN,RELATION:KV']
GAAP Gross Profit is $212539 T in second quarter 2023 @@@	NER:[('GAAP', 'METRIC'), ('Gross Profit', 'METRIC'), ('$212539 T', 'MONEY'), ('second quarter', 'CALENDAR')]###RELATIONS:['METRIC:GAAP GROSS PROFIT,METRICTYPE:REGULAR,MONEY:$212.5MN,RELATION:KV']
GAAP Gross Margin is 81.74% in second quarter 2023 @@@	NER:[('GAAP', 'METRIC'), ('Gross Margin', 'METRIC'), ('81.74%', 'PCT'), ('second quarter', 'CALENDAR')]###RELATIONS:['METRIC:GAAP GROSS MARGIN,METRICTYPE:REGULAR,PCT:81

Story continues.	NER:[]###RELATIONS:[]
SCBQ*** Business Highlights and Recent Developments.	NER:[]###RELATIONS:[]
PG*** The metrics listed below identified as BILL standalone exclude the results of Divvy and Invoice2go.	NER:[('Divvy', 'ORG')]###RELATIONS:[]
PG*** Completed the acquisition of Finmark, a financial planning and cash flow insights software company.	NER:[('acquisition', 'CHG'), ('Finmark', 'ORG')]###RELATIONS:[]
PG*** Served 435,800 businesses using our solutions as of the end of the second quarter.	NER:[('435,800', 'CD'), ('businesses', 'METRIC'), ('second quarter', 'CALENDAR')]###RELATIONS:['METRIC:TOTAL NUMBER OF PAID CUSTOMERS,METRICTYPE:OPMETRIC,CD:435800,RELATION:KV']
This includes 182,700 BILL standalone customers, 24,700 spending businesses that used Divvy, and 228,500 subscribers that used Invoice2go.	NER:[('182,700', 'CD'), ('BILL', 'CD'), ('standalone customers', 'METRIC'), ('24,700', 'CD'), ('spending businesses', 'METRIC'), ('Divvy', 'ORG'), ('228,500', 'CD'), 

PG*** Q1 Total Revenue Increased 94% Year-Over-Year.	NER:[('Q1', 'CALENDAR'), ('Revenue', 'METRIC'), ('Increased', 'CHG'), ('94%', 'PCT'), ('Year-Over-Year', 'CALENDAR')]###RELATIONS:[]
PG*** SAN JOSE, Calif., November 03, 2022--(BUSINESS WIRE)--BILL (NYSE: BILL), a leader in financial automation software for small and midsize businesses (SMBs), today announced financial results for the first fiscal quarter ended September 30, 2022.	NER:[('SAN JOSE', 'LOC'), ('Calif.', 'LOC'), ('November 03, 2022', 'DATE'), ('BILL', 'SYM'), ('leader', 'METRIC'), ('small and midsize businesses', 'MISC'), ('SMBs', 'MISC'), ('announced', 'CHG'), ('first fiscal quarter', 'CALENDAR'), ('September 30, 2022', 'DATE')]###RELATIONS:[]
PG***	NER:[]###RELATIONS:[]
SCHQ*** Financial Highlights for the First Quarter of Fiscal 2023:.	NER:[('SCHQ', 'PER'), ('First Quarter', 'CALENDAR')]###RELATIONS:[]
PG*** The financial measures listed below identified as BILL standalone exclude the results of Divvy and Invoice2go.	

PG*** Processed 19.6 million transactions during the fourth quarter, an increase of 45% year-over-year.	NER:[('Processed', 'CHG'), ('19.6 million', 'CD'), ('transactions', 'METRIC'), ('fourth quarter', 'CALENDAR'), ('increase', 'CHG'), ('45%', 'PCT'), ('year-over-year', 'CALENDAR')]###RELATIONS:[]
>>> This includes 10.8 million transactions on our BILL standalone platform, representing an increase of 23% year-over-year, and 8.5 million Divvy card transactions, an increase of 83% year-over-year.	NER:[('10.8 million', 'CD'), ('transactions', 'METRIC'), ('BILL standalone platform', 'MISC'), ('increase', 'CHG'), ('23%', 'PCT'), ('year-over-year', 'CALENDAR'), ('8.5 million', 'CD'), ('Divvy card transactions', 'MISC'), ('increase', 'CHG'), ('83%', 'PCT'), ('year-over-year', 'CALENDAR')]###RELATIONS:[]
PG*** Added Google Cloud Chief Marketing Officer, Alison Wagonfeld, to our board of directors.	NER:[('Added', 'CS'), ('Google Cloud', 'MISC'), ('Chief Marketing Officer', 'ROLE'), ('Alison Wag

...

PG*** These statements are forward-looking and actual results may differ materially.	NER:[]###RELATIONS:[]
Refer to the Forward-Looking Statements safe harbor below for information on the factors that could cause our actual results to differ materially from these forward-looking statements.	NER:[]###RELATIONS:[]
PG*** Guidance for non-GAAP income from operations excludes stock-based compensation expense and related employer payroll taxes, amortization expense of acquired intangible assets, amortization of debt discount and issuance costs.	NER:[]###RELATIONS:[]
As a result of the adoption of ASU 2020-06 on August 1, 2022, guidance for non-GAAP net income per share uses the if-converted method to calculate the potentially diluted shares related to the convertible senior notes.	NER:[('August 1, 2022', 'DATE'), ('guidance', 'FC'), ('non-GAAP', 'METRIC'), ('net income per share', 'METRIC')]###RELATIONS:[]
Accordingly, we are required to add back the non-GAAP interest expense to our non-GAAP

PG*** Cash flow: Cash provided by operations was $128.5 million, or 36% of revenue, compared to $93.3 million, or 40% of revenue, in the first quarter of fiscal 2022.	NER:[('Cash provided by operations', 'METRIC'), ('$128.5 million', 'MONEY'), ('36%', 'PCT'), ('revenue', 'METRIC'), ('$93.3 million', 'MONEY'), ('40%', 'PCT'), ('revenue', 'METRIC'), ('first quarter', 'CALENDAR'), ('2022', 'YEAR')]###RELATIONS:['METRIC:OPERATING CASH FLOW,METRICTYPE:REGULAR,MONEY:$128.5MN,RELATION:KV']
Free cash flow was $95.6 million, or 27% of revenue, compared to $83.4 million, or 36% of revenue, in the first quarter of fiscal 2022.	NER:[('Free cash flow', 'METRIC'), ('$95.6 million', 'MONEY'), ('27%', 'PCT'), ('revenue', 'METRIC'), ('$83.4 million', 'MONEY'), ('36%', 'PCT'), ('revenue', 'METRIC'), ('first quarter', 'CALENDAR'), ('2022', 'YEAR')]###RELATIONS:['METRIC:FREE CASH FLOW,METRICTYPE:REGULAR,MONEY:$95.6MN,RELATION:KV', 'METRIC:GAAP FREE CASH FLOW,METRICTYPE:REGULAR,MONEY:$95.6MN,RELATION:KV']


PG*** Non-GAAP net income per share of $1.23 to $1.25, assuming approximately 157 million common shares outstanding using the "if-converted" method for our senior convertible notes.	NER:[('Non-GAAP', 'METRIC'), ('net income per share', 'METRIC'), ('$1.23', 'MONEY'), ('$1.25', 'MONEY'), ('157 million', 'CD'), ('shares', 'METRIC')]###RELATIONS:['METRIC:EPS,METRICTYPE:REGULAR,MONEY:$1.23,RANGEM1:$1.25,RELATION:KVRANGE']
PG*** These statements are forward-looking and actual results may differ materially.	NER:[]###RELATIONS:[]
Refer to the Forward-Looking Statements safe harbor below for information on the factors that could cause our actual results to differ materially from these forward-looking statements.	NER:[]###RELATIONS:[]
PG*** Guidance for non-GAAP income from operations excludes stock-based compensation expense and related employer payroll taxes, amortization expense of acquired intangible assets, amortization of debt discount and issuance costs.	NER:[]###RELATIONS:[]
As a result 

PG*** Cash flow: Cash provided by operations was $103.1 million, or 32% of revenue, compared to $44.7 million, or 23% of revenue, in the fourth quarter of fiscal 2021.	NER:[('Cash provided by operations', 'METRIC'), ('$103.1 million', 'MONEY'), ('32%', 'PCT'), ('revenue', 'METRIC'), ('$44.7 million', 'MONEY'), ('23%', 'PCT'), ('revenue', 'METRIC'), ('fourth quarter', 'CALENDAR'), ('2021', 'YEAR')]###RELATIONS:['METRIC:OPERATING CASH FLOW,METRICTYPE:REGULAR,MONEY:$103.1MN,RELATION:KV']
Free cash flow was $74.8 million, or 24% of revenue, compared to $27.7 million, or 14% of revenue, in the fourth quarter of fiscal 2021.	NER:[('Free cash flow', 'METRIC'), ('$74.8 million', 'MONEY'), ('24%', 'PCT'), ('revenue', 'METRIC'), ('$27.7 million', 'MONEY'), ('14%', 'PCT'), ('revenue', 'METRIC'), ('fourth quarter', 'CALENDAR'), ('2021', 'YEAR')]###RELATIONS:['METRIC:FREE CASH FLOW,METRICTYPE:REGULAR,MONEY:$74.8MN,RELATION:KV', 'METRIC:GAAP FREE CASH FLOW,METRICTYPE:REGULAR,MONEY:$74.8MN,RELATION:K

PG*** Recently Issued Accounting Pronouncements.	NER:[]###RELATIONS:[]
PG*** Effective August 1, 2022, the beginning of our fiscal year ending July 31, 2023, we adopted the ASU 2020-06, regarding ASC Topic 470 “Debt” and ASC Topic 815 “Derivatives and Hedging” (“ASU 2020-06”), which simplifies the accounting for convertible debt.	NER:[('August 1, 2022', 'DATE'), ('July 31, 2023', 'DATE')]###RELATIONS:[]
ASU 2020-06 amended the calculation of diluted earnings per share for certain convertible debt instruments, among other changes.	NER:[]###RELATIONS:[]
As a result of the adoption of the ASU 2020-06, we are required to use the “if-converted” method to calculate the non-GAAP net income per diluted share related to our senior convertible notes, which assumes conversion at the beginning of the reporting period, with settlement entirely in shares of common stock, unless the results would be anti-dilutive.	NER:[]###RELATIONS:[]
Accordingly, to account for our potentially diluted shares relate

In [120]:
# Format 2
source = "../../Summary/DATA/ENTITYRELATION/Backup/Format-1"
files = glob.glob(source+"/*_ER.tsv")
#print(files)
for file in files:
    basefile = os.path.basename(file)
    #print(basefile)
    outfile = "../../Summary/DATA/ENTITYRELATION/Train/"+basefile
    of = None
    #print(outfile)
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            continue
        
        of = open(outfilePath, "w", encoding = "utf-8")
        #of.write("filename\tSentence1\tSentence2\n")
        
    with open(file, "r", encoding = "utf-8") as f:
        line = f.readline()
        #print(line)
        while line:
            if("NER" not in line):
                of.write(line)
            else:
                nsplit = line.split("\t")
                nline = nsplit[2]
                nline = nline.split("###")[1]
                nsplit[2] = nline
                nline = "\t".join(nsplit)
                of.write(nline)
            line = f.readline()
    if of:
        of.close()
    #break

Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2022-08-04_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2022-11-03_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-02-16_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-08-03_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-08-18_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-11-03_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-02-02_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-05-04_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-08-17_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYREL

In [58]:
# Format 3
source = "../../Summary/DATA/ENTITYRELATION/Backup/Format-1"
files = glob.glob(source+"/*_ER.tsv")
#print(files)
for file in files:
    #print(file)
    basefile = os.path.basename(file)
    #print(basefile)
    outfile = "../../Summary/DATA/ENTITYRELATION/Train/"+basefile
    of = None
    #print(outfile)
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            continue
        
        print("Creating Output File {}".format(outfile))
        of = open(outfilePath, "w", encoding = "utf-8")
        
    with open(file, "r", encoding = "utf-8") as f:
        line = f.readline()
        #print(line)
        while line:
            if("NER" not in line):
                of.write(line)
            else:
                relations = dict()
                relations["RELATIONS"] = list()
                nsplit = line.split("\t")
                nline = nsplit[2]
                nline = nline.split("###")[1]
                #nline = "{"+nline+"}"
                #print(nline)
                nline = nline.replace("RELATIONS:[","").replace("]","").replace("\n","").replace("\'","")
                relations["RELATIONS"] = (nline.split(", "))
                nline = json.dumps(relations)
                #print(nline)
                nsplit[2] = nline
                nline = "\t".join(nsplit)
                of.write(nline)
            line = f.readline()
    if of:
        of.close()
    #break

Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2022-08-04_EP_YH_ER.tsv already exists
Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2022-11-03_EP_YH_ER.tsv already exists
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-02-16_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-08-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-08-18_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-11-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-02-02_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-05-04_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-08-17_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/CFLT_2022-08-03_EP_YH_ER.ts

In [14]:
# Format 6
maxFiles = 60
fileCnt = 0
source = "../../Summary/DATA/ENTITYRELATION/Backup/Format-6"
files = glob.glob(source+"/*_ER.tsv")
#print(files)
for file in files:
    #print(file)
    basefile = os.path.basename(file)
    #print(basefile)
    outfile = "../../Summary/DATA/ENTITYRELATION/Train/"+basefile
    of = None
    #print(outfile)
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            continue
        
        if(maxFiles > 0):
            fileCnt = fileCnt + 1
            if(fileCnt > maxFiles):
                break
            
        print("Creating Output File {}".format(outfile))
        of = open(outfilePath, "w", encoding = "utf-8")
        
        
    with open(file, "r", encoding = "utf-8") as f:
        line = f.readline()
        #print(line)
        while line:
            if ("RELATIONS" not in line):
                of.write(line)
                #of.write("\n")
            else:
                relations = dict()
                nsplit = line.split("\t")
                relation = nsplit[2]
                relation = relation.replace("\n","")
                sent = nsplit[1]
                sent = sent.replace("\t","")
                #print(sent)
                relation = json.loads(relation)
                #print(relation)
                sentences = getSentences(None, nlp, sent)
                nerl = None
                for l in sentences:
                    text1 = list()
                    text1.append(l)
                    for doc in nlp.pipe(text1, disable=["tagger"]):
                        for ent in doc.ents:
                            if(not nerl):
                                nerl = ent.label_.replace(":","").replace(",","")+"="+ent.text.replace(":","").replace(",","")
                            else:
                                nerl = nerl + "," + ent.label_.replace(":","").replace(",","")+"="+ent.text.replace(":","").replace(",","")
                #relations["ENTITIES"] = None
                relations["ENTITIES"] = list()
                relations["ENTITIES"].append(nerl)
                rlist = list()
                for rstr in relation["RELATIONS"]:
                    rstr = rstr.replace(",MONEY","!!MONEY").replace(",TYPE","!!TYPE").replace(",CD","!!CD").replace(",RELATION","!!RELATION").replace(",PCT","!!PCT").replace(",DATE","!!DATE").replace(",CALENDAR","!!CALENDAR")
                    rlist.append(rstr)
                relations["RELATIONS"] = rlist
                #print(relations)
                nsplit[2] = json.dumps(relations)
                nsplit[1] = sent
                nline = "\t".join(nsplit)
                #nline = nline + "</s>" #Not Required
                of.write(nline)
                of.write("\n")
            line = f.readline()
    if(of):
        of.close()
    #break

Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2022-11-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-02-16_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-08-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-08-18_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-11-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-02-02_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-05-04_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-08-17_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/CFLT_2022-08-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/CFLT_2022-11-02_EP_YH_ER.tsv
Creating O

In [19]:
# Format 7
maxFiles = -1
fileCnt = 0
source = "../../Summary/DATA/ENTITYRELATION/Backup/Format-6"
files = glob.glob(source+"/*_ER.tsv")
#print(files)
for file in files:
    #print(file)
    basefile = os.path.basename(file)
    #print(basefile)
    outfile = "../../Summary/DATA/ENTITYRELATION/Train/"+basefile
    of = None
    #print(outfile)
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            continue
        
        if(maxFiles > 0):
            fileCnt = fileCnt + 1
            if(fileCnt > maxFiles):
                break
            
        print("Creating Output File {}".format(outfile))
        of = open(outfilePath, "w", encoding = "utf-8")
        
        
    with open(file, "r", encoding = "utf-8") as f:
        line = f.readline()
        #print(line)
        while line:
            if ("RELATIONS" not in line):
                of.write(line)
                #of.write("\n")
            else:
                relations = dict()
                nsplit = line.split("\t")
                relation = nsplit[2]
                relation = relation.replace("\n","")
                sent = nsplit[1]
                sent = sent.replace("\t","")
                #print(sent)
                relation = json.loads(relation)
                #print(relation)
                rlist = list()
                for rstr in relation["RELATIONS"]:
                    rstr = rstr.replace("METRIC:","KEY:").replace("RELATION:","LINK:").replace("TYPE:OPERATIONAL","TYPE:INPUT").replace("TYPE:REGULAR","TYPE:OUT")
                    rlist.append(rstr)
                relations["RELATIONS"] = rlist
                #print(relations)
                nsplit[2] = json.dumps(relations)
                nsplit[1] = sent
                nline = "\t".join(nsplit)
                #nline = nline + "</s>" #Not Required
                of.write(nline)
                of.write("\n")
            line = f.readline()
    if(of):
        of.close()
    #break

Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2022-11-03_EP_YH_ER.tsv already exists
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-02-16_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-05-09_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/APPN_2023-08-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-08-18_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2022-11-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-02-02_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-05-04_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/BILL_2023-08-17_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/CFLT_2022-08-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/CFLT_2022-11-02_EP_YH_ER.tsv
Crea

Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/ZM_2023-05-22_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/ZM_2023-08-21_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/ZS_2022-09-08_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/ZS_2022-12-01_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/ZS_2023-03-03_EP_YH_ER.tsv
Creating Output File ../../Summary/DATA/ENTITYRELATION/Train/ZS_2023-06-01_EP_YH_ER.tsv


In [14]:
devDataFile = "../../Summary/DATA/ENTITYRELATION/Dev/dev.tsv"
trainDataFile = "../../Summary/DATA/ENTITYRELATION/Train/train.tsv"
testDataFile = "../../Summary/DATA/ENTITYRELATION/Test/test.tsv"

def writeTrainingData(writeFile, writeDir):
    files = glob.glob(writeDir+"/*_ER.tsv")
    print(files)
    frames = list()

    if(len(files) > 0):
        for file in files:
            print(file)
            #df = pd.read_csv(file, sep="\t", encoding = "utf-8").astype(str)
            df = pd.read_csv(file, sep="\t", encoding = "ISO-8859-1").astype(str)
            df = df.dropna()
            df = df[df['Sentence1'].notna()]
            #print(df)
            frames.append(df)
    result = pd.concat(frames)
    print(result)
    result.to_csv(writeFile, sep='\t', index=False, header=True)

In [22]:
writeTrainingData(trainDataFile, trainDir)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


['../../Summary/DATA/ENTITYRELATION/Train\\APPN_2023-02-16_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\APPN_2023-05-09_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\APPN_2023-08-03_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\BILL_2022-08-18_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\BILL_2023-02-02_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\BILL_2023-05-04_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\BILL_2023-08-17_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\CFLT_2022-08-03_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\CFLT_2022-11-02_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\CFLT_2023-05-03_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\CFLT_2023-08-02_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\CRWD_2022-08-30_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\CRWD_2022-11-29_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Train\\CRWD_202

../../Summary/DATA/ENTITYRELATION/Train\OTHER_JS23_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PATH_2022-09-06_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PATH_2022-12-01_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PATH_2023-05-24_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PAYC_2022-08-02_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PAYC_2022-11-01_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PAYC_2023-05-02_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PAYC_2023-08-01_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PLTR_2022-08-08_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PLTR_2022-11-07_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PLTR_2023-02-13_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\PLTR_2023-05-08_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\RNG_2022-11-09_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\RNG_2023-02-15_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Train\R

In [23]:
writeTrainingData(devDataFile, devDir)

['../../Summary/DATA/ENTITYRELATION/Dev\\APPN_2022-11-03_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\BILL_2022-11-03_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\CFLT_2023-01-30_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\CRWD_2023-05-31_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\DDOG_2023-08-08_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\DOCN_2022-11-07_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\DOCU_2022-12-08_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\HUBS_2023-05-03_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\MDB_2023-08-31_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\NET_2022-08-04_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\OKTA_2022-11-30_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\OTHER_DEV_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\OTHER_JS23_DEV_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Dev\\PATH_2023-03-15_EP_YH_ER.tsv', '../../Summary

In [24]:
writeTrainingData(testDataFile, testDir)

['../../Summary/DATA/ENTITYRELATION/Test\\FIVN_2023-02-22_EP_YH_ER.tsv', '../../Summary/DATA/ENTITYRELATION/Test\\S_2022-12-06_EP_YH_ER.tsv']
../../Summary/DATA/ENTITYRELATION/Test\FIVN_2023-02-22_EP_YH_ER.tsv
../../Summary/DATA/ENTITYRELATION/Test\S_2022-12-06_EP_YH_ER.tsv
                 filename                                          Sentence1  \
0   FIVN_2023-02-22_EP_YH  GAAP Net Loss Per Share is $(0.19) in fourth q...   
1   FIVN_2023-02-22_EP_YH  Cash And Cash Equivalents is $180520 T in four...   
2   FIVN_2023-02-22_EP_YH  GAAP Gross Profit is $112051 T in fourth quart...   
3   FIVN_2023-02-22_EP_YH  GAAP Gross Margin is 53.78% in fourth quarter ...   
4   FIVN_2023-02-22_EP_YH  GAAP Free Cash Flow is $36.6MN in fourth quart...   
..                    ...                                                ...   
34     S_2022-12-06_EP_YH  PG*** These statements are forward-looking and...   
35     S_2022-12-06_EP_YH  Refer to the below for information on the fact...   
36   

In [47]:
import sklearn
import logging
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [48]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [49]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    )

In [50]:
model_args = Seq2SeqArgs()
#model_args = T5Args()
model_args.do_sample = True
model_args.train_batch_size = 4
model_args.use_multiprocessing = False
model_args.num_train_epochs = 2
#model_args.learning_rate = 5e-5
#model_args.no_save = True
model_args.output_dir = "seroutputs/"

model_args.eval_batch_size = 4
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training_steps = 600

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model_args.max_length = 128
#model_args.max_seq_length = 32
model_args.num_return_sequences = 10
model_args.top_k = 30
model_args.top_p = 0.95
model_args.fp16 = False
#model_args.num_beams = None

model_args.use_early_stopping = False
#model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "mcc"
#model_args.early_stopping_metric_minimize = False
#model_args.early_stopping_patience = 5
#model_args.evaluate_during_training_steps = 500
#model_args.wandb_project =  "visualization-demo"

model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
    use_cuda=True,
)

#model = T5Model("t5", "flan-t5-base", args=model_args, use_cuda=True)

# Train the model
model.train_model(
    train_df, eval_data=eval_df, matches=count_matches
)
results = model.eval_model(eval_df)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/1964 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/491 [00:00<?, ?it/s]

...

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into seroutputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to seroutputs/.
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/570 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/143 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/143 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.008569831715624412, 'matches': 0}


In [51]:
model_args = Seq2SeqArgs()
#model_args = T5Args()
model_args.do_sample = True
model_args.train_batch_size = 4
model_args.use_multiprocessing = False
model_args.num_train_epochs = 2
#model_args.learning_rate = 5e-5
#model_args.no_save = True
model_args.output_dir = "seroutputs/"

model_args.eval_batch_size = 4
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training_steps = 1200

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model_args.max_length = 128
#model_args.max_seq_length = 32
model_args.num_return_sequences = 10
model_args.top_k = 30
model_args.top_p = 0.95
model_args.fp16 = False
#model_args.num_beams = None

model_args.use_early_stopping = False

In [57]:
model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="seroutputs", args=model_args
)

print(
    model.predict(
        [
            "Total revenue is expected to be between 1.2 BN and 1.3 BN"
        ]
    )
)

seroutputs\model_args.json


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[["RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MISSION:$0.3 BN", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MATCH:$0.3 BNRELATION:KVRANGE']", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MOST:$2.3 BN", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MISSION:KV']", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MEX:$0.3 BNRELATION:KVRANGE']", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MATCH:$0.3 BN", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MELF:$0.3 BNRELATION:KVRANGE']", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MOST BN", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MARK:$3.3 BNRELATION:KVRANGE']", "RELATIONS:['METRIC:REVENUE,TYPE:REGULAR,MONEY:$1.2 BN,MELF:$0.3 BNRELATION:KVRANGE']"]]


In [17]:
train_df = pd.read_csv(trainDataFile, sep="\t", encoding = "utf-8").astype(str)
eval_df = pd.read_csv(devDataFile, sep="\t", encoding = "utf-8").astype(str)
test_df = pd.read_csv(testDataFile, sep="\t", encoding = "utf-8").astype(str)

train_df = train_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
test_df = test_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)

#train_df = train_df[["input_text", "target_text"]]
#eval_df = eval_df[["input_text", "target_text"]]
#test_df = test_df[["input_text", "target_text"]]

train_df["prefix"] = "get relations"
train_df = train_df[["prefix", "input_text", "target_text"]]
#train_df = train_df[["input_text", "target_text"]]

eval_df["prefix"] = "get relations"
eval_df = eval_df[["prefix", "input_text", "target_text"]]
#eval_df = eval_df[["input_text", "target_text"]]

test_df["prefix"] = "get relations"
test_df = test_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
train_df = train_df[train_df['input_text'].notna()]

eval_df = eval_df.dropna()
eval_df = eval_df[eval_df['input_text'].notna()]

test_df = test_df.dropna()
test_df = test_df[test_df['input_text'].notna()]

#train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
#train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
print("TRAIN DATA ..............")
print(train_df)

#eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
#eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)
print("EVAL DATA ..............")
print(eval_df)

print("TEST DATA ..............")
print(test_df)

TRAIN DATA ..............
             prefix                                         input_text  \
0     get relations  GAAP Net Loss Per Share is $(0.47) in fourth q...   
1     get relations  Cash And Cash Equivalents is $148132 T in four...   
2     get relations  GAAP Gross Profit is $90555 T in fourth quarte...   
3     get relations  GAAP Gross Margin is 71.99% in fourth quarter ...   
4     get relations                                              PG***   
...             ...                                                ...   
5278  get relations  We have not reconciled our expectations to non...   
5279  get relations  For those reasons, we are also unable to addre...   
5280  get relations  Accordingly, a reconciliation for the guidance...   
5281  get relations  PG*** In the third quarter of fiscal 2023, we ...   
5282  get relations  PG*** For further information regarding why we...   

                                            target_text  
0     {"RELATIONS": ["KEY:G

In [15]:
import logging

import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

In [16]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [20]:
# This model is being used
train_params = {
    #'max_seq_length': 512,
    #'model_max_length': 512,
    'max_length': 1024,
    #'min_length': 784,
    #'max_new_tokens': 1024,
    #'min_new_tokens': 784,
    'output_dir':"teroutputs/",
    'train_batch_size': 2,
    'eval_batch_size': 2,
    'num_train_epochs': 2,
    'learning_rate': 3e-4,
    'evaluate_generated_text': True,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 2500,
    'use_multiprocessing': False,
    'fp16': False,
    'save_steps': -1,
    'save_eval_checkpoints': False,
    'save_model_every_epoch': False,
    'no_cache': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'preprocess_inputs': False,
    'top_k': 50,
    'top_p': 0.95,
    'num_beams': 4,
    'num_return_sequences': 1
    #'config': {
    #            'min_length': 784,
    #            'max_length': 1024,
    #            "early_stopping": True,
    #            'length_penalty': 0.80,
    #            'no_repeat_ngram_size': 3,
    #          }
}
model = T5Model('t5', 't5-base', args=train_params)
#model.train_model(train, eval_data=valid)

def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

model.train_model(train_df, eval_data=eval_df, matches=count_matches)

print(model.eval_model(eval_df, matches=count_matches))

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\transformers\models\t5\tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/5283 [00:00<?, ?it/s]

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\transformers\tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2642 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/731 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/731 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1461 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.02240532503836696, 'matches': 0}


['{"RELATIONS": ["KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.61!!LINK:KV"]}', '{"RELATIONS": ["KEY:Cash And Cash Equivalents!!TYPE:OUT!!MONEY:$51 MN!!LINK:KV"]}', '{"RELATIONS": ["KEY:GAAP GROSS PROFIT!!TYPE:OUT!!MONEY:$84 MN!!LINK:KV"]}', '{"RELATIONS": ["KEY:GAAP GROSS MARGIN!!TYPE:OUT!!PCT:71.36%!!LINK:KV"]}', '{"RELATIONS": [""]}', '{"RELATIONS": [""]}', '{"RELATIONS": ["KEY:SUBSCRIPTION REVENUE-Cloud!!TYPE:INPUT!!MONEY:$60.6 MN!!LINK:KV"]}', '{"RELATIONS": [""]}', '{"RELATIONS": [""]}', '{"RELATIONS": [""]}', '{"RELATIONS": [""]}', '{"RELATIONS": ["KEY:SUBSCRIPTION REVENUE-Cloud!!TYPE:INPUT!!MONEY:$60.6 MN!!LINK:KV"]}', '{"RELATIONS": ["KEY:SUBSCRIPTIONS REVENUE!!TYPE:INPUT!!MONEY:$86.5 MN!!LINK:KV"]}', '{"RELATIONS": ["KEY:PROFESSIONAL SERVICES REVENUE!!TYPE:OUT!!MONEY:$31.4 MN!!LINK:KV"]}', '{"RELATIONS": ["KEY:REVENUE!!TYPE:OUT!!MONEY:$117.9 MN!!LINK:KV"]}', '{"RELATIONS": ["KEY:Cloud subscription revenue retention rate!!TYPE:INPUT!!PCT:115.0%!!LINK:KV"]}', '{"RELATIONS

In [17]:
pred_params = {
        'max_seq_length': 512,
        'max_length': 1024,
        'use_multiprocessed_decoding': False,
        'top_k': 50,
        'top_p': 0.95,
        'num_return_sequences': 1
        }

to_predict = [
    "get relations: Organic core revenue was $114.9 million, up 71% year-over-year and Non-GAAP net income was $12 million. "
]

model = T5Model('t5', 'teroutputs', args=pred_params)
predictions = model.predict(to_predict)
print(predictions)
pred = (predictions[0])
print(pred, len(pred))
pred = "{"+pred+"}"
js = json.loads(pred)
#js = ast.literal_eval(pred)
print((js))

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\transformers\tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['"RELATIONS": ["KEY:organic core revenue!!TYPE:INPUT!!MONEY:$114.9 MN!!LINK:KV", "KEY:NON-GAAP NET INCOME!!TYPE:OUT!!MONEY:$12 MN!!LINK:KV"]']
"RELATIONS": ["KEY:organic core revenue!!TYPE:INPUT!!MONEY:$114.9 MN!!LINK:KV", "KEY:NON-GAAP NET INCOME!!TYPE:OUT!!MONEY:$12 MN!!LINK:KV"] 139
{'RELATIONS': ['KEY:organic core revenue!!TYPE:INPUT!!MONEY:$114.9 MN!!LINK:KV', 'KEY:NON-GAAP NET INCOME!!TYPE:OUT!!MONEY:$12 MN!!LINK:KV']}


In [35]:
testFile = "../../Summary/Refined/APPN/APPN_2023-02-16_EP_YH.txt"
with open(testFile, "r", encoding = "utf-8") as f:
    line = f.readline()
    #print(line)
    while line:
        #print(line)
        if("ED***" in line):
            break
        elif("CS***" in line or "TBLST***" in line or "TBLET***" in line):
            #print(line+"\t"+"RELATION:None")
            line = f.readline()
            continue
        #elif("PG***" in line or "NOPAD***" in line or ("***" not in line)):
        else:
            sentences = getSentences(None, nlp, line)
            for l in sentences:
                #print(l)
                pline, isProcess = preProcessSent(l)
                if not isProcess:
                    continue
                text1 = list()
                text1.append(l)
                #nerl = list()
                #for doc in nlp.pipe(text1, disable=["tagger"]):
                #    nerl = ([(ent.text, ent.label_) for ent in doc.ents])
                to_predict = ["get relations: "+pline]
                predictions = model.predict(to_predict)
                pred = (predictions[0][0])
                print(pline+"\t"+pred)
                #pred = "{"+pred+"}"
                #print(pline)
                
        line = f.readline()

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

GAAP Net Loss Per Share is $(0.47) in fourth quarter 2022 @@@	"ENTITIES": ["METRIC:GAAP Net Loss Per Share,MONEY:$(0.47),CALENDAR:fourth quarter,YEAR:2022"], "RELATIONS": ["METRIC:GAAP net income per share,TYPE:REGULAR,MONEY:-$0.47,RELATION:KV"]


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Cash And Cash Equivalents is $148132 T in fourth quarter 2022 @@@	"ENTITIES": ["METRIC:Cash,METRIC:Cash Equivalents,MONEY:$1481322,CALENDAR:fourth quarter,YEAR:2022,METRIC:CASH"], "RELATIONS": ["METRIC:CASH AND EQUIVALENTS,TYPE:REGULAR,MONEY:$1482 MN,RELATION:KV"]


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

Total revenue is expected to be between $530.0 million and $535.0 million, representing a year-over-year increase of 13% to 14%.	"ENTITIES": ["METRIC:revenue,MONEY:$530.0 million,MONEY:$535.0 million,CALENDAR:year-over-year,CHG:increase,PCT:13%,PCT:14%"], "RELATIONS": ["METRIC:REVENUE,TYPE:REGULAR,MONEY:$530.0 MN,MONEY:$535.0 MN,RELATION:KVRANGE"]


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Adjusted EBITDA loss is expected to be between $(75.0) million and $(70.0) million.	"ENTITIES": ["METRIC:Adjusted EBITDA loss,MONEY:$(75.0) million,MONEY:$(70.0) million"], "RELATIONS": ["METRIC:ADJUSTED EBITDA,TYPE:REGULAR,MONEY:-$75.0 MN,MONEY:-$70.0 MN,RELATION:KVRANGE"]


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Non-GAAP net loss per share is expected to be between $(1.14) and $(1.07), assuming weighted average common shares outstanding of 73.2 million.	"ENTITIES": ["METRIC:Non-GAAP net loss per share,MONEY:$(1.14),MONEY:$(1.07),METRIC:shares outstanding,CD:73.2 million"], "RELATIONS": ["METRIC:Non-GAAP net income per share,TYPE:REGULAR,MONEY:-$1.14,MONEY:-$1.07,RELATION:KVRANGE"]


In [18]:
def writetofile(inputfile, to_predict, predictions, of, forTrain=False):
    if forTrain:
        return
    for to_pred, preds in zip(to_predict, predictions):
        #print(preds)
        pred = preds
        pred = pred.replace("\n","")
        pred = "{"+pred+"}"
        topred = to_pred.replace("get relations: ","")
        #print(pred, str(type(ast.literal_eval(pred))))
        outstr = ""
        try:
            if(str(type(ast.literal_eval(pred))) == "<class 'dict'>"):
                outstr = (topred + "\t" + pred)
            else:
                outstr = (topred + "\t" + "NO PREDICTION")
        except:
            outstr = (topred + "\t" + "NO PREDICTION")

        #print(outstr)
        #print("\n")
        outstr = inputfile + "\t" + outstr
        of.write(outstr)
        of.write("\n\n")

In [19]:
maxCount = 30

def createDatawithModel(origFile, csym, nlp, erModel, forTrain=True):
    basefile = os.path.basename(origFile)
    inputfile = os.path.splitext(basefile)[0]
    if(not forTrain):
        #print("NOT SUPPORTED WITHOUT TRAINING FLAG AS OF NOW")
        #return
        if not isScanningRqd(origFile):
            print("Format not supported for file {}".format(origFile))
            return False
        
        outdir = "../../Summary/PostRefinedV2"
        outfileDir = outdir+"/"+csym
        if not os.path.exists(outfileDir):
            os.makedirs(outfileDir)
        outfilePath = outfileDir+"/"+inputfile+".txt"
        #print(outfilePath)
        outfile = Path(outfilePath)
        if outfile.is_file():
            print(outfilePath + " Already exists")
            return False
        
        print("Creating post refined data file " + str(outfile))
        of = None
        of = open(outfile, "w", encoding = "utf-8")
        of.write("filename\tOriginal\tPrediction\n")
        
        with open(origFile, "r", encoding = "utf-8") as f:
            line = f.readline()
            #print(line)
            to_predict = list()
            while line:
                #print(line)
                if("ED***" in line):
                    break
                elif("CS***" in line or "TBLST***" in line or "TBLET***" in line):
                    if(len(to_predict) > 0):
                        #print(to_predict)
                        #print("\n\n")
                        predictions = erModel.predict(to_predict)
                        #print(predictions)
                        #print("\n\n")
                        writetofile(inputfile, to_predict, predictions, of, forTrain=False)
                    to_predict = list()
                    line = line.replace("\n","")
                    #print(line + " #### " + "NOT PROCCESSED")
                    #print("\n")
                    sentences = getSentences(None, nlp, line)
                    for l in sentences:
                        if("CS***" in line):
                            if("CS***" in l):
                                of.write(inputfile + "\t" + l + "\t" + "NO PREDICTION")
                            else:
                                of.write(inputfile + "\t" + "CS*** "+l + "\t" + "NO PREDICTION")
                        else:
                            of.write(inputfile + "\t" + l + "\t" + "NO PREDICTION")
                        of.write("\n\n")
                    line = f.readline()
                    continue
                #elif("PG***" in line or "NOPAD***" in line or ("***" not in line)):
                else:
                    sentences = getSentences(None, nlp, line)
                    #to_predict = list()
                    for l in sentences:
                        #print(l)
                        pline, isProcess = preProcessSent(l)
                        if not isProcess:
                            continue
                        to_predict = to_predict + ["get relations: "+l]
                        if(len(to_predict) >= maxCount):
                            #print(to_predict)
                            #print("\n\n")
                            predictions = erModel.predict(to_predict)
                            #print(predictions)
                            #print("\n\n")
                            writetofile(inputfile, to_predict, predictions, of, forTrain=False)
                            to_predict = list()
                        #of.write(inputfile+"\t"+pline+"\t"+pred)
                        #of.write("\n\n")
                line = f.readline()
            
            if(len(to_predict) > 0):
                #print(to_predict)
                #print("\n\n")
                predictions = erModel.predict(to_predict)
                #print(predictions)
                #print("\n\n")
                writetofile(inputfile, to_predict, predictions, of, forTrain=False)
                to_predict = list()
        if(of):
            of.close()
    else:
        if not isScanningRqd(origFile):
            print("Format not supported for file {}".format(origFile))
            return
        
        trainfilePath = trainDir+"/"+inputfile+"_ER.tsv"
        devfilePath = devDir+"/"+inputfile+"_ER.tsv"
        testfilePath = testDir+"/"+inputfile+"_ER.tsv"
        
        trainoutfile = Path(trainfilePath)
        devoutfile = Path(devfilePath)
        testoutfile = Path(testfilePath)
        
        if(not trainoutfile.is_file() and not devoutfile.is_file() and not testoutfile.is_file()):
            print("Creating Train file " + str(trainoutfile))
            of = None
            of = open(trainoutfile, "w", encoding = "utf-8")
            of.write("filename\tSentence1\tSentence2\n")
            
            with open(origFile, "r", encoding = "utf-8") as f:
                line = f.readline()
                #print(line)
                while line:
                    #print(line)
                    if("ED***" in line):
                        break
                    elif("CS***" in line or "TBLST***" in line or "TBLET***" in line):
                        #print(line+"\t"+"RELATION:None")
                        line = f.readline()
                        continue
                    #elif("PG***" in line or "NOPAD***" in line or ("***" not in line)):
                    else:
                        sentences = getSentences(None, nlp, line)
                        for l in sentences:
                            #print(l)
                            pline, isProcess = preProcessSent(l)
                            if not isProcess:
                                continue
                            to_predict = ["get relations: "+l]
                            predictions = erModel.predict(to_predict)
                            pred = (predictions[0])
                            pred = pred.replace("\n","")
                            pred = "{"+pred+"}"
                            of.write(inputfile+"\t"+l+"\t"+pred)
                            of.write("\n\n")
                    line = f.readline()
            if(of):
                of.close()
    return True

In [22]:
# Create relations using gen AI model
entPath = "../../Summary/entities/"
rPath = "../../Summary/Refined/"
files = glob.glob(entPath+"/*-ENTITIES.json")   
if(len(files) > 0):
    for file in (files):
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        csym = filename.split("-")[0]
        cPath = rPath + "/" + csym
        #print(cPath)
        cfiles = glob.glob(cPath+"/*.txt")
        #print(cfiles)
        if(len(cfiles) > 0):
            for cf in cfiles:
                if isScanningRqd(cf):
                    created = createDatawithModel(cf, csym, nlp, model, forTrain=False)
                    if not created:
                        continue
                    break
        if created:
            break

../../Summary/PostRefinedV2/APPN/APPN_2022-08-04_EP_YH.txt Already exists
../../Summary/PostRefinedV2/APPN/APPN_2022-11-03_EP_YH.txt Already exists
../../Summary/PostRefinedV2/APPN/APPN_2023-02-16_EP_YH.txt Already exists
../../Summary/PostRefinedV2/APPN/APPN_2023-05-09_EP_YH.txt Already exists
../../Summary/PostRefinedV2/APPN/APPN_2023-08-03_EP_YH.txt Already exists
../../Summary/PostRefinedV2/APPN/APPN_2023-11-02_EP_YH.txt Already exists
../../Summary/PostRefinedV2/BILL/BILL_2022-08-18_EP_YH.txt Already exists
../../Summary/PostRefinedV2/BILL/BILL_2022-11-03_EP_YH.txt Already exists
../../Summary/PostRefinedV2/BILL/BILL_2023-02-02_EP_YH.txt Already exists
../../Summary/PostRefinedV2/BILL/BILL_2023-05-04_EP_YH.txt Already exists
../../Summary/PostRefinedV2/BILL/BILL_2023-08-17_EP_YH.txt Already exists
../../Summary/PostRefinedV2/BILL/BILL_2023-11-02_EP_YH.txt Already exists
../../Summary/PostRefinedV2/CFLT/CFLT_2022-08-03_EP_YH.txt Already exists
../../Summary/PostRefinedV2/CFLT/CFLT_

Generating outputs:   0%|          | 0/5 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/6 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/12 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

In [49]:
entPath = "../../Summary/entities/"
rPath = "../../Summary/Refined/"
files = glob.glob(entPath+"/*-ENTITIES.json")   
if(len(files) > 0):
    for file in (files):
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        csym = filename.split("-")[0]
        #print(csym)
        cPath = rPath + "/" + csym
        #print(cPath)
        cfiles = glob.glob(cPath+"/*.txt")
        if(len(cfiles) > 0):
            for cf in cfiles:
                if isScanningRqd(cf):
                    createDatawithModel(cf, csym, nlp, model, forTrain=True)

Creating Train file ..\..\Summary\DATA\ENTITYRELATION\Train\CRWD_2023-08-30_EP_YH_ER.tsv


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [54]:
tokenizer.tokenize("get relations: GAAP Free Cash Flow is $443MN in second quarter 2024 @@@.")

['▁get',
 '▁relations',
 ':',
 '▁GA',
 'AP',
 '▁Free',
 '▁Cash',
 '▁',
 'Flow',
 '▁is',
 '▁$4',
 '43',
 'M',
 'N',
 '▁in',
 '▁second',
 '▁quarter',
 '▁20',
 '24',
 '▁@',
 '@',
 '@',
 '.']

In [47]:
inp = "../../Summary/PostRefinedV2/BILL/BILL_2022-08-18_EP_YH.txt"
no_pred = "NO PREDICTION"
df = pd.read_csv(inp, sep="\t", encoding = "utf-8").astype(str)
df = df.dropna()
df = df[df['Original'].notna()]
sentences = list(df['Original'])
entities = dict()
entities["ORG"] = list()
entities["MISC"] = list()
entities["SYM"] = list()
entities["CALENDAR"] = list()
entities["DATE"] = list()
entities["METRICS"] = list()
entities["OPMETRICS"] = list()

#print(df["Original"][0])
#print(type(sentences))
#for sent in sentences:
#    print(sent)

for orig, pred in zip(df['Original'], df['Prediction']):
    if(pred != no_pred):
        print(orig, pred)
        pdict = ast.literal_eval(pred)
        #print(pdict["ENTITIES"], pdict["RELATIONS"])
        if ("ENTITIES" in pdict):
            ents = pdict["ENTITIES"][0]
            #print(ents)
            esplit = (ents.split(","))
            #print(esplit)
            lner = ([(es.split("=")[1], es.split("=")[0], 0, 1) for es in esplit])
            for es in esplit:
                mtype = es.split("=")[0]
                mval = es.split("=")[1].upper()
                if(mtype == "MISC" and mval not in entities["MISC"]):
                    entities["MISC"].append(mval)
                elif(mtype == "ORG" and mval not in entities["ORG"]):
                    entities["ORG"].append(mval)
                elif(mtype == "SYM" and mval not in entities["SYM"]):
                    entities["SYM"].append(mval)
                elif(mtype == "DATE" and mval not in entities["DATE"]):
                    entities["DATE"].append(mval)
                elif(mtype == "CALENDAR" and mval not in entities["CALENDAR"]):
                    entities["CALENDAR"].append(mval)
            #print(lner)
        relation = pdict["RELATIONS"]
        #print(relation)
        if(len(relation)) > 0 and relation[0] != '':
            for rl in relation:
                rs = rl.split("!!")
                #print(len(rs))
                key = rs[0].split(":")[1]
                print("METRIC="+key)
                if key.upper() not in entities["METRICS"]:
                    entities["METRICS"].append(key.upper())
                if key not in entities:
                    entities[key] = list()
                otype = rs[1].split(":")[1]
                #print(otype)
                seq = ([(rs[i].split(":")[1], rs[i].split(":")[0], 0, 1) for i in range(2,len(rs))])
                seq.append((otype, "METRICTYPE", 0, 1))
                if(otype == "INPUT" and key.upper() not in entities["OPMETRICS"]):
                    entities["OPMETRICS"].append(key.upper())
                #print(seq)
                entities[key].append(seq)

if("ORG" in entities):
    entities["ORG"] = list(set(entities["ORG"]))
if("MISC" in entities):
    entities["MISC"] = list(set(entities["MISC"]))
if("CALENDAR" in entities):
    entities["CALENDAR"] = list(set(entities["CALENDAR"]))
if("DATE" in entities):
    entities["DATE"] = list(set(entities["DATE"]))
if("SYM" in entities):
    entities["SYM"] = list(set(entities["SYM"]))
print(entities)
print("\n\n")

print(entities["METRICS"])
print("\n\n")

print(entities["OPMETRICS"])
            

GAAP Net Loss Per Share is $(0.81) in fourth quarter 2022 @@@ {"RELATIONS": ["KEY:GAAP net income per share!!TYPE:OUT!!MONEY:-$0.81!!LINK:KV"]}
METRIC=GAAP net income per share
Cash And Cash Equivalents is $1596542 T in fourth quarter 2022 @@@ {"RELATIONS": ["KEY:CASH AND EQUIVALENTS!!TYPE:OUT!!MONEY:$1.59 BN!!LINK:KV"]}
METRIC=CASH AND EQUIVALENTS
GAAP Gross Profit is $156780 T in fourth quarter 2022 @@@ {"RELATIONS": ["KEY:GAAP GROSS PROFIT!!TYPE:OUT!!MONEY:$156 MN!!LINK:KV"]}
METRIC=GAAP GROSS PROFIT
GAAP Gross Margin is 78.3% in fourth quarter 2022 @@@ {"RELATIONS": ["KEY:GAAP GROSS MARGIN!!TYPE:OUT!!PCT:78.3%!!LINK:KV"]}
METRIC=GAAP GROSS MARGIN
GAAP Free Cash Flow is $(12.1MN) in fourth quarter 2022 @@@ {"RELATIONS": ["KEY:GAAP FREE CASH FLOW!!TYPE:OUT!!MONEY:-$12.1 MN!!LINK:KV"]}
METRIC=GAAP FREE CASH FLOW
GAAP Free Cash Flow Margin is (6.04)% in fourth quarter 2022 @@@ {"RELATIONS": ["KEY:GAAP FREE CASH FLOW MARGIN!!TYPE:OUT!!PCT:(6.04%)!!LINK:KV"]}
METRIC=GAAP FREE CASH FLOW M